<a href="https://colab.research.google.com/github/arjasc5231/moodots/blob/ACRNN/SER/CNN/ADRNN/ADRNN_ver1_dilatedCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Speech Emotion Recognition From 3D Log-Mel Spectrograms With Deep Learning network를 바탕으로 구현.  

다음 논문의 특징 중  


1.   ACRNN
2.   dilated cnn
3.   batch normalization
4.   leaky relu
5.   residual
6.   center loss
  
1, 2만 우선 구현

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import sys
sys.path.append("/content/drive/MyDrive/team_runner/colab/emoDB/CNN")
from kfold import kfold

In [ ]:
X_train, X_test, Y_train, Y_test = np.load("/content/drive/MyDrive/team_runner/colab/dataset/emoDB/emo_mel_3d.npy", allow_pickle=True)
X = np.concatenate((X_train, X_test), axis=0)
Y = np.concatenate((Y_train, Y_test), axis=0)

print(X.shape)
print(Y.shape)

(911, 128, 128, 3)
(911,)


In [ ]:
def create_model():
    inputs = keras.Input(shape=(128, 128, 3))
    conv1 = keras.layers.Conv2D(filters=32, kernel_size=[3, 3], padding='same', activation=tf.nn.relu)(inputs)
    pool1 = keras.layers.MaxPool2D()(conv1)
    
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=[3, 3], dilation_rate=2, padding='same', activation=tf.nn.relu)(pool1)
    conv3 = keras.layers.Conv2D(filters=128, kernel_size=[3, 3], dilation_rate=2, padding='same', activation=tf.nn.relu)(conv2)
    conv4 = keras.layers.Conv2D(filters=256, kernel_size=[3, 3], dilation_rate=2, padding='same', activation=tf.nn.relu)(conv3)

    trans = keras.layers.Permute((2,1,3))(conv4)
    reshape = keras.layers.Reshape((-1, 64*128))(trans) # 열 개수(freq축)*ch
    linear = keras.layers.Dense(512)(reshape)
    """ # 기존보다 time축으로 4배 길어졌는데, 때문에 시간이 너무 오래걸리고, 빠르게 overfitting되어 성능도 60%정도
    lstm = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=False, dropout=0.5))(linear)

    fc = keras.layers.Dense(128)(lstm)
    """
    lstm = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True, dropout=0.5))(linear) # (time=15,hidden=256)

    attention_score1 = keras.layers.Dense(1, activation='tanh')(lstm) # lstm(time,hidden)*W(hidden,1)=score(time,1)
    attention_score2 = keras.layers.Softmax()(attention_score1)
    attention = keras.layers.Dot(axes=(1,1))([lstm, attention_score2]) # (time=15, hidden=256) * (time,) => (hidden=256)
    flatten = keras.layers.Flatten()(attention)
  
    fc = keras.layers.Dense(128)(flatten)
    # 윗 주석은 CRNN, 다음 코드는 ACRNN
    drop = keras.layers.Dropout(0.5)(fc)
    output = keras.layers.Dense(7)(drop)
    return keras.Model(inputs=inputs, outputs=output)

In [ ]:
model = create_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 64, 64, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 64)   18496       max_pooling2d[0][0]              
______________________________________________________________________________________________

In [ ]:
kfold(5, create_model, X, Y, 0.001, 100, 120, 10) # 정확도 75%정도. 너무 오래걸려서 2epoch만 보고 끊음
# fold당 25분정도

Learning started. k=5
<<fold 1>>
Epoch	loss		train acc	test acc
10  	2.0763  	0.1963  	0.2143
20  	1.6252  	0.4011  	0.4378
30  	1.3840  	0.5043  	0.4688
40  	1.1896  	0.5532  	0.5029
50  	0.9033  	0.7066  	0.6020
60  	0.7413  	0.7741  	0.6383
70  	0.5573  	0.8448  	0.5811
80  	0.3128  	0.9268  	0.6663
90  	0.3541  	0.9025  	0.5942
100  	0.2756  	0.9150  	0.5992
110  	0.0373  	1.0000  	0.6543
120  	0.0141  	1.0000  	0.6653
max train accuracy: 1.0000
max test accuracy: 0.6964
<<fold 2>>
Epoch	loss		train acc	test acc
10  	1.7045  	0.3744  	0.3857
20  	1.1714  	0.5856  	0.6127
30  	1.0187  	0.6686  	0.6171
40  	0.8596  	0.7173  	0.5899
50  	0.5996  	0.8289  	0.6521
60  	0.4304  	0.8894  	0.6210
70  	0.2000  	0.9538  	0.7015
80  	0.0933  	0.9913  	0.6532
90  	0.0720  	0.9950  	0.6737
100  	0.1044  	0.9887  	0.6604
110  	0.0093  	1.0000  	0.6898
120  	0.0026  	1.0000  	0.7159
max train accuracy: 1.0000
max test accuracy: 0.7259
<<fold 3>>
Epoch	loss		train acc	test acc


KeyboardInterrupt: ignored